In [23]:
import pandas as pd


ratings = pd.read_csv(r'C:\Users\mystm\OneDrive - Maryville University\Desktop\Movie Review Database\ml-32m1\ml-32m\ratings.csv')
print("First 20 rows of ratings.csv:")
print(ratings.head(20))

movies = pd.read_csv(r'C:\Users\mystm\OneDrive - Maryville University\Desktop\Movie Review Database\ml-32m1\ml-32m\movies.csv')
print("\nFirst 20 rows of movies.csv:")
print(movies.head(20))


First 20 rows of ratings.csv:
    userId  movieId  rating  timestamp
0        1       17     4.0  944249077
1        1       25     1.0  944250228
2        1       29     2.0  943230976
3        1       30     5.0  944249077
4        1       32     5.0  943228858
5        1       34     2.0  943228491
6        1       36     1.0  944249008
7        1       80     5.0  944248943
8        1      110     3.0  943231119
9        1      111     5.0  944249008
10       1      161     1.0  943231162
11       1      166     5.0  943228442
12       1      176     4.0  944079496
13       1      223     3.0  944082810
14       1      232     5.0  943228442
15       1      260     5.0  943228696
16       1      302     4.0  944253272
17       1      306     5.0  944248888
18       1      307     5.0  944253207
19       1      322     4.0  944053801

First 20 rows of movies.csv:
    movieId                                  title  \
0         1                       Toy Story (1995)   
1         2  

In [4]:
#merge the data

# Merge the datasets on movieId
merged_df = pd.merge(ratings, movies, on='movieId')

# Display first 10 rows
print(merged_df.head(10))

   userId  movieId  rating  timestamp  \
0       1       17     4.0  944249077   
1       1       25     1.0  944250228   
2       1       29     2.0  943230976   
3       1       30     5.0  944249077   
4       1       32     5.0  943228858   
5       1       34     2.0  943228491   
6       1       36     1.0  944249008   
7       1       80     5.0  944248943   
8       1      110     3.0  943231119   
9       1      111     5.0  944249008   

                                               title  \
0                       Sense and Sensibility (1995)   
1                           Leaving Las Vegas (1995)   
2  City of Lost Children, The (Cité des enfants p...   
3  Shanghai Triad (Yao a yao yao dao waipo qiao) ...   
4          Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   
5                                        Babe (1995)   
6                            Dead Man Walking (1995)   
7        White Balloon, The (Badkonake sefid) (1995)   
8                                  Bravehear

In [7]:
#Check for unique numbers
print(f"Number of unique users: {merged_df['userId'].nunique()}")
print(f"Number of unique movies: {merged_df['movieId'].nunique()}")

Number of unique users: 200948
Number of unique movies: 84432


In [9]:
#Missing values

print("Missing values in dataset:")
print(merged_df.isnull().sum())

Missing values in dataset:
userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64


In [11]:
#Most rated movie
most_rated = merged_df.groupby('title')['rating'].count().sort_values(ascending=False)
print("Top 10 most rated movies:")
print(most_rated.head(10))

Top 10 most rated movies:
title
Shawshank Redemption, The (1994)                             102929
Forrest Gump (1994)                                          100296
Pulp Fiction (1994)                                           98409
Matrix, The (1999)                                            93808
Silence of the Lambs, The (1991)                              90330
Star Wars: Episode IV - A New Hope (1977)                     85010
Fight Club (1999)                                             77332
Jurassic Park (1993)                                          75233
Schindler's List (1993)                                       73849
Lord of the Rings: The Fellowship of the Ring, The (2001)     73122
Name: rating, dtype: int64


In [13]:
#Highest rated movie
average_ratings = merged_df.groupby('title')['rating'].mean()
rating_counts = merged_df.groupby('title')['rating'].count()

# Filter for movies with at least 50 ratings
popular_movies = average_ratings[rating_counts >= 50].sort_values(ascending=False)

print("Top 10 highest-rated movies (min 50 ratings):")
print(popular_movies.head(10))

Top 10 highest-rated movies (min 50 ratings):
title
Planet Earth II (2016)              4.446830
Planet Earth (2006)                 4.444369
Band of Brothers (2001)             4.426539
Shawshank Redemption, The (1994)    4.404614
Cosmos                              4.330081
Godfather, The (1972)               4.317030
Parasite (2019)                     4.312254
Blue Planet II (2017)               4.300086
Twin Peaks (1989)                   4.298684
Twelve Angry Men (1954)             4.286192
Name: rating, dtype: float64


In [22]:
#Data cleaning
#Drop duplicates
merged_df = merged_df.drop_duplicates()

#Convert timestamp to datetime format (Optional, useful for time-based analysis)
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'], unit='s')

#Drop timestamp column if not needed
merged_df = merged_df.drop(columns=['timestamp'])

print("Cleaned dataset preview:")
print(merged_df.head(10))

KeyError: 'timestamp'

In [26]:
print("Cleaned dataset preview:")
print(merged_df.head(20))

Cleaned dataset preview:
    userId  movieId  rating  \
0        1       17     4.0   
1        1       25     1.0   
2        1       29     2.0   
3        1       30     5.0   
4        1       32     5.0   
5        1       34     2.0   
6        1       36     1.0   
7        1       80     5.0   
8        1      110     3.0   
9        1      111     5.0   
10       1      161     1.0   
11       1      166     5.0   
12       1      176     4.0   
13       1      223     3.0   
14       1      232     5.0   
15       1      260     5.0   
16       1      302     4.0   
17       1      306     5.0   
18       1      307     5.0   
19       1      322     4.0   

                                                title  \
0                        Sense and Sensibility (1995)   
1                            Leaving Las Vegas (1995)   
2   City of Lost Children, The (Cité des enfants p...   
3   Shanghai Triad (Yao a yao yao dao waipo qiao) ...   
4           Twelve Monkeys (a.k.a. 12 

In [11]:
#Content Based Filtering Method
#Converting the genres into machine-readable format
#Using TF-IDF Vectorization

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

#Load the movies dataset
movies = pd.read_csv(r'C:\Users\mystm\OneDrive - Maryville University\Desktop\Movie Review Database\ml-32m1\ml-32m\movies.csv')

#Fill NaN values (if any)
movies['genres'] = movies['genres'].fillna('')

#Initialize the TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

#Fit and transform the genres column
genre_matrix = tfidf.fit_transform(movies['genres'])

#Convert to a DataFrame for easy viewing
genre_df = pd.DataFrame(genre_matrix.toarray(), index=movies['title'], columns=tfidf.get_feature_names_out())

#Display first few rows
print(genre_df.head())


                                    action  adventure  animation  children  \
title                                                                        
Toy Story (1995)                       0.0   0.458969   0.478003  0.480576   
Jumanji (1995)                         0.0   0.551895   0.000000  0.577877   
Grumpier Old Men (1995)                0.0   0.000000   0.000000  0.000000   
Waiting to Exhale (1995)               0.0   0.000000   0.000000  0.000000   
Father of the Bride Part II (1995)     0.0   0.000000   0.000000  0.000000   

                                      comedy  crime  documentary     drama  \
title                                                                        
Toy Story (1995)                    0.282693    0.0          0.0  0.000000   
Jumanji (1995)                      0.000000    0.0          0.0  0.000000   
Grumpier Old Men (1995)             0.596953    0.0          0.0  0.000000   
Waiting to Exhale (1995)            0.534581    0.0          0.

In [13]:
#Computing Movie Similarity
#Consine Similarity: measures how close two movies are based on their genre vectors

from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(genre_matrix, genre_matrix)

# Convert to a DataFrame for easier indexing
movie_sim_df = pd.DataFrame(cosine_sim, index=movies['title'], columns=movies['title'])

# Display the first few rows
print(movie_sim_df.head())


title                               Toy Story (1995)  Jumanji (1995)  \
title                                                                  
Toy Story (1995)                            1.000000        0.831624   
Jumanji (1995)                              0.831624        1.000000   
Grumpier Old Men (1995)                     0.168754        0.000000   
Waiting to Exhale (1995)                    0.151122        0.000000   
Father of the Bride Part II (1995)          0.282693        0.000000   

title                               Grumpier Old Men (1995)  \
title                                                         
Toy Story (1995)                                   0.168754   
Jumanji (1995)                                     0.000000   
Grumpier Old Men (1995)                            1.000000   
Waiting to Exhale (1995)                           0.895517   
Father of the Bride Part II (1995)                 0.596953   

title                               Waiting to Exhale

In [15]:
# Function recommending movies based on genre similarity.
def recommend_movies_by_genre(movie_title, movie_sim_df, top_n=5):
    # Get the similarity scores for the given movie
    sim_scores = movie_sim_df[movie_title]
    
    # Sort the movies based on similarity scores in descending order
    sim_scores = sim_scores.sort_values(ascending=False)
    
    # Exclude the input movie from the recommendations
    sim_scores = sim_scores.drop(movie_title)
    
    # Get the top_n most similar movies
    top_recommendations = sim_scores.head(top_n)
    
    return top_recommendations

# Example usage: Recommend 5 movies similar to "Toy Story (1995)"
recommended_movies = recommend_movies_by_genre('Toy Story (1995)', movie_sim_df, top_n=5)
print(recommended_movies)


title
UglyDolls (2019)                                  1.0
Emperor's New Groove, The (2000)                  1.0
Onward (2020)                                     1.0
Trolls World Tour (2020)                          1.0
Adventures of Rocky and Bullwinkle, The (2000)    1.0
Name: Toy Story (1995), dtype: float64


In [21]:
#Confirming that the movies do have similar genres
# Get the index of the movies you want to inspect
toy_story_idx = movies[movies['title'] == 'Toy Story (1995)'].index[0]
ugly_dolls_idx = movies[movies['title'] == 'UglyDolls (2019)'].index[0]
emperor_new_groove_idx = movies[movies['title'] == 'Emperor\'s New Groove, The (2000)'].index[0]

# Access the genre vector for each movie
toy_story_vector = genre_matrix[toy_story_idx].toarray().flatten()
ugly_dolls_vector = genre_matrix[ugly_dolls_idx].toarray().flatten()
emperor_new_groove_vector = genre_matrix[emperor_new_groove_idx].toarray().flatten()

# Print the vectors to compare
print(toy_story_vector)
print(ugly_dolls_vector)
print(emperor_new_groove_vector)



[0.         0.45896914 0.47800272 0.48057644 0.282693   0.
 0.         0.         0.49999168 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]
[0.         0.45896914 0.47800272 0.48057644 0.282693   0.
 0.         0.         0.49999168 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]
[0.         0.45896914 0.47800272 0.48057644 0.282693   0.
 0.         0.         0.49999168 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]
